In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
import pandas_gbq
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Race Results

In [4]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [5]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP=GP[(GP["Position"]!=0)&(GP["Position"]<21)]
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,30,L LAWSON,LAW,lawson,RB,6692FF,rb,Liam,Lawson,Liam Lawson,None,NZL,9,9,19,NaT,NaT,NaT,0 days 00:01:10.563000,Finished,2,2024,United States Grand Prix
1,22,Y TSUNODA,TSU,tsunoda,RB,6692FF,rb,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,14,14,10,NaT,NaT,NaT,NaT,+1 Lap,0,2024,United States Grand Prix
2,27,N HULKENBERG,HUL,hulkenberg,Haas F1 Team,B6BABD,haas,Nico,Hulkenberg,Nico Hulkenberg,https://media.formula1.com/d_driver_fallback_i...,GER,8,8,11,NaT,NaT,NaT,0 days 00:01:02.957000,Finished,4,2024,United States Grand Prix
3,20,K MAGNUSSEN,MAG,kevin_magnussen,Haas F1 Team,B6BABD,haas,Kevin,Magnussen,Kevin Magnussen,https://media.formula1.com/d_driver_fallback_i...,DEN,11,11,8,NaT,NaT,NaT,0 days 00:01:19.782000,Finished,0,2024,United States Grand Prix
4,10,P GASLY,GAS,gasly,Alpine,0093cc,alpine,Pierre,Gasly,Pierre Gasly,https://media.formula1.com/d_driver_fallback_i...,FRA,12,12,6,NaT,NaT,NaT,0 days 00:01:30.558000,Finished,0,2024,United States Grand Prix


In [6]:
GP_Race_Wins=pd.pivot_table(GP[GP["Year"].isin(range(1900,2025))],index=['FullName'], columns=['Position'], values='Points', aggfunc=['count'])
GP_Race_Wins=GP_Race_Wins.fillna(0)
GP_Race_Wins.head()

count                                                         \
Position         1    2    3    4    5    6    7    8    9    10   11   12   
FullName                                                                     
Adolf Brudes    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
Adolfo Cruz     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
Adrian Sutil    0.0  0.0  0.0  1.0  3.0  3.0  5.0  5.0  8.0  5.0  8.0  2.0   
Adrián Campos   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
Aguri Suzuki    0.0  0.0  1.0  0.0  0.0  4.0  3.0  4.0  2.0  3.0  2.0  3.0   

                                                            
Position         13   14   15    16    17    18    19   20  
FullName                                                    
Adolf Brudes    0.0  0.0  0.0   1.0   0.0   0.0   0.0  0.0  
Adolfo Cruz     0.0  0.0  0.0   1.0   0.0   0.0   0.0  0.0  
Adrian Sutil   13.0  6.0  7.0  10.0  11.0  10.0  12.0  6.0  
Adrián Campos   0.0  2.0  1.0   2.0   1.0   0.0   2.0  0.0  
Aguri Suzuki    2.0  2.0  1.0   5.0   2.0   4.0   4.0  2.0

In [7]:
#Remove the column that says full name
GP_Race_Wins.columns = GP_Race_Wins.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Race_Wins=GP_Race_Wins.reset_index()

GP_Race_Wins=GP_Race_Wins.sort_values(by=["FullName"],ascending=[True])
GP_Race_Wins.head()

Position,FullName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Adolf Brudes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Adolfo Cruz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Adrian Sutil,0.0,0.0,0.0,1.0,3.0,3.0,5.0,5.0,8.0,5.0,8.0,2.0,13.0,6.0,7.0,10.0,11.0,10.0,12.0,6.0
3,Adrián Campos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,0.0,2.0,0.0
4,Aguri Suzuki,0.0,0.0,1.0,0.0,0.0,4.0,3.0,4.0,2.0,3.0,2.0,3.0,2.0,2.0,1.0,5.0,2.0,4.0,4.0,2.0


In [8]:
import plotly.graph_objects as go

fig = go.Figure(go.Table
 (header={"values": GP_Race_Wins.columns},
  cells=dict(
    values=GP_Race_Wins.T.values,
    # fill_color=[GP_Race_Wins.Color],
    align='center', font=dict(color='Black', size=9)
    ))
 )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Race_Wins.T.values
#                                 if c == "All"
#                                 else GP_Race_Wins.loc[GP_Race_Wins["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Race_Wins["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig.update_layout(
    title="Race Results By Position and Count",
        title_x=0.5,
        template="plotly_white",

    yaxis = dict(tickfont = dict(size=11)),
    xaxis = dict(tickfont = dict(size=11)),
    font=dict(
        family="PT Sans Narrow",
        size=11,
        color="Black"
    ),
    height=1000,
    width=1400,
    title_font_family="PT Sans Narrow",
    margin=dict(l=10, r=10, t=35, b=10),
)
fig.show()

In [9]:
fig.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Race Results By Position and Count.html",full_html=False, include_plotlyjs='cdn')

In [10]:
#Sprint Results

In [11]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_sprint`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq3 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [12]:
GP_sprint=df_bq3
GP_sprint=GP_sprint.fillna(0)
GP_sprint["Position"]=GP_sprint["Position"].astype(int)
GP_sprint["GridPosition"]=GP_sprint["GridPosition"].astype(int)
GP_sprint["Points"]=GP_sprint["Points"].astype(int)
GP_sprint=GP_sprint[(GP_sprint["Position"]!=0)&(GP_sprint["Position"]<21)]
GP_sprint.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,9,0,10,NaT,NaT,NaT,0 days 00:29:44.191000,Finished,0,2021,São Paulo Grand Prix
1,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,13,0,14,NaT,NaT,NaT,0 days 00:28:37.451000,Finished,0,2021,Italian Grand Prix
2,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,10,0,13,NaT,NaT,NaT,0 days 00:26:25.821000,Finished,0,2021,British Grand Prix
3,31,E OCON,OCO,OCO,Alpine,2293D1,0,Esteban,Ocon,Esteban Ocon,0,0,9,0,9,NaT,NaT,NaT,0 days 00:25:20.843000,Finished,0,2023,Belgian Grand Prix
4,31,E OCON,OCO,OCO,Alpine,2293D1,0,Esteban,Ocon,Esteban Ocon,0,0,7,0,8,NaT,NaT,NaT,0 days 00:31:03.332000,Finished,2,2023,Austrian Grand Prix


In [13]:
#Get the counts by final position in race

In [14]:
GP_Sprint_Wins=pd.pivot_table(GP_sprint[GP_sprint["Year"].isin(range(1957,2025))],index=['FullName'], columns=['Position'], values='Points', aggfunc=['count'])
GP_Sprint_Wins=GP_Sprint_Wins.fillna(0)
GP_Sprint_Wins.head()

count                                                    \
Position              1    2    3    4    5    6    7    8    9    10   11   
FullName                                                                     
Alexander Albon      0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  2.0  0.0  0.0   
Antonio Giovinazzi   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
Carlos Sainz         0.0  2.0  3.0  2.0  4.0  2.0  1.0  1.0  0.0  0.0  1.0   
Charles Leclerc      0.0  4.0  1.0  3.0  2.0  2.0  2.0  0.0  0.0  0.0  0.0   
Daniel Ricciardo     0.0  0.0  1.0  1.0  0.0  2.0  0.0  0.0  1.0  1.0  3.0   

                                                                 
Position             12   13   14   15   16   17   18   19   20  
FullName                                                         
Alexander Albon     1.0  2.0  0.0  1.0  1.0  3.0  1.0  0.0  1.0  
Antonio Giovinazzi  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  
Carlos Sainz        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Charles Leclerc     2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Daniel Ricciardo    2.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0

In [15]:
#Remove the column that says full name
GP_Sprint_Wins.columns = GP_Sprint_Wins.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Sprint_Wins=GP_Sprint_Wins.reset_index()

GP_Sprint_Wins=GP_Sprint_Wins.sort_values(by=["FullName"],ascending=[True])
GP_Sprint_Wins.head()

Position,FullName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Alexander Albon,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,3.0,1.0,0.0,1.0
1,Antonio Giovinazzi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Carlos Sainz,0.0,2.0,3.0,2.0,4.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Charles Leclerc,0.0,4.0,1.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Daniel Ricciardo,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,1.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
import plotly.graph_objects as go

fig_sprint = go.Figure(go.Table
 (header={"values": GP_Sprint_Wins.columns},
  cells=dict(
    values=GP_Sprint_Wins.T.values,
    # fill_color=[GP_Sprint_Wins.Color],
    align='center', font=dict(color='Black', size=9)
    ))
 )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Sprint_Wins.T.values
#                                 if c == "All"
#                                 else GP_Sprint_Wins.loc[GP_Sprint_Wins["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Sprint_Wins["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_sprint.update_layout(
    title="Sprint Race Results By Position and Count",
        title_x=0.5,
        template="plotly_white",

    yaxis = dict(tickfont = dict(size=11)),
    xaxis = dict(tickfont = dict(size=11)),
    font=dict(
        family="PT Sans Narrow",
        size=11,
        color="Black"
    ),
    height=800,
    width=1400,
    title_font_family="PT Sans Narrow",
    margin=dict(l=10, r=10, t=35, b=10),
)
fig_sprint.show()

In [17]:
fig_sprint.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Sprint Race Results By Position and Count.html",full_html=False, include_plotlyjs='cdn')

In [18]:
#Team Race Results

In [19]:
#Get the counts by final position in race

In [20]:
GP_Race_Wins_Team=pd.pivot_table(GP[GP["Year"].isin(range(1900,2025))],index=['TeamName'], columns=['Position'], values='Points', aggfunc=['count'])
GP_Race_Wins_Team=GP_Race_Wins_Team.fillna(0)
GP_Race_Wins_Team.head()

count                                                        \
Position             1    2    3    4    5    6    7     8     9     10    11   
TeamName                                                                        
AFM                 0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   1.0   0.0   0.0   
AGS                 0.0  0.0  0.0  0.0  0.0  2.0  1.0   3.0   4.0   3.0   1.0   
ATS                 0.0  0.0  0.0  0.0  3.0  3.0  2.0   6.0   4.0   4.0   9.0   
Alfa Romeo         11.0  8.0  9.0  8.0  8.0  7.0  7.0  11.0  15.0  20.0  20.0   
Alfa Romeo Racing   0.0  0.0  0.0  1.0  1.0  0.0  3.0   4.0   7.0   8.0  12.0   

                                                                         
Position             12    13    14    15    16    17    18    19    20  
TeamName                                                                 
AFM                 0.0   0.0   2.0   0.0   0.0   0.0   1.0   0.0   0.0  
AGS                 3.0   4.0   1.0   2.0   4.0   2.0   1.0   3.0   2.0  
ATS                10.0   9.0   7.0   7.0  10.0   4.0  12.0   3.0   4.0  
Alfa Romeo         25.0  21.0  10.0  19.0  23.0  18.0  17.0  15.0  15.0  
Alfa Romeo Racing  11.0  12.0  16.0  18.0   9.0   7.0   6.0   3.0   2.0

In [21]:
#Remove the column that says full name
GP_Race_Wins_Team.columns = GP_Race_Wins_Team.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Race_Wins_Team=GP_Race_Wins_Team.reset_index()

GP_Race_Wins_Team=GP_Race_Wins_Team.sort_values(by=["TeamName"],ascending=[True])
GP_Race_Wins_Team.head()

Position,TeamName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,AFM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
1,AGS,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,4.0,3.0,1.0,3.0,4.0,1.0,2.0,4.0,2.0,1.0,3.0,2.0
2,ATS,0.0,0.0,0.0,0.0,3.0,3.0,2.0,6.0,4.0,4.0,9.0,10.0,9.0,7.0,7.0,10.0,4.0,12.0,3.0,4.0
3,Alfa Romeo,11.0,8.0,9.0,8.0,8.0,7.0,7.0,11.0,15.0,20.0,20.0,25.0,21.0,10.0,19.0,23.0,18.0,17.0,15.0,15.0
4,Alfa Romeo Racing,0.0,0.0,0.0,1.0,1.0,0.0,3.0,4.0,7.0,8.0,12.0,11.0,12.0,16.0,18.0,9.0,7.0,6.0,3.0,2.0


In [22]:
import plotly.graph_objects as go

fig_team = go.Figure(go.Table
 (header={"values": GP_Race_Wins_Team.columns},
  cells=dict(
    values=GP_Race_Wins_Team.T.values,
    # fill_color=[GP_Race_Wins_Team.Color],
    align='center', font=dict(color='Black', size=9)
    ))
 )

# fig_team.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Race_Wins_Team.T.values
#                                 if c == "All"
#                                 else GP_Race_Wins_Team.loc[GP_Race_Wins_Team["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Race_Wins_Team["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_team.update_layout(
    title="Team Race Results By Position and Count",
        title_x=0.5,
        template="plotly_white",

    yaxis = dict(tickfont = dict(size=1)),
    xaxis = dict(tickfont = dict(size=11)),
    font=dict(
        family="PT Sans Narrow",
        size=11,
        color="Black"
    ),
    height=810,
    width=1400,
    title_font_family="PT Sans Narrow",
    margin=dict(l=10, r=10, t=35, b=10)
)
fig_team.show()

In [23]:
fig_team.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Team Race Results By Position and Count.html",full_html=False, include_plotlyjs='cdn')

In [24]:
#Team Sprint Race Results

In [25]:
GP_Sprint_Wins_Team=pd.pivot_table(GP_sprint[GP_sprint["Year"].isin(range(1900,2025))],index=['TeamName'], columns=['Position'], values='Points', aggfunc=['count'])
GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.fillna(0)
GP_Sprint_Wins_Team.head()

count                                                    \
Position             1    2    3    4    5    6    7    8    9    10   11   
TeamName                                                                    
Alfa Romeo          0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  2.0  0.0   
Alfa Romeo Racing   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
AlphaTauri          0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  2.0  1.0   
Alpine              0.0  0.0  1.0  0.0  0.0  1.0  3.0  0.0  5.0  1.0  3.0   
Aston Martin        0.0  0.0  0.0  1.0  1.0  1.0  0.0  3.0  1.0  3.0  2.0   

                                                                
Position            12   13   14   15   16   17   18   19   20  
TeamName                                                        
Alfa Romeo         1.0  2.0  2.0  2.0  2.0  2.0  0.0  2.0  2.0  
Alfa Romeo Racing  0.0  2.0  0.0  1.0  0.0  0.0  2.0  0.0  0.0  
AlphaTauri         3.0  0.0  2.0  3.0  3.0  3.0  1.0  1.0  2.0  
Alpine             2.0  4.0  2.0  4.0  1.0  2.0  2.0  0.0  1.0  
Aston Martin       2.0  5.0  3.0  2.0  1.0  1.0  1.0  2.0  3.0

In [26]:
#Remove the column that says full name
GP_Sprint_Wins_Team.columns = GP_Sprint_Wins_Team.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.reset_index()

GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.sort_values(by=["TeamName"],ascending=[True])
GP_Sprint_Wins_Team.head()

Position,TeamName,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,Alfa Romeo,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0
1,Alfa Romeo Racing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0
2,AlphaTauri,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,1.0,3.0,0.0,2.0,3.0,3.0,3.0,1.0,1.0,2.0
3,Alpine,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,1.0,3.0,2.0,4.0,2.0,4.0,1.0,2.0,2.0,0.0,1.0
4,Aston Martin,0.0,0.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,3.0,2.0,2.0,5.0,3.0,2.0,1.0,1.0,1.0,2.0,3.0


In [27]:
import plotly.graph_objects as go

fig_sprint_team = go.Figure(go.Table
 (header={"values": GP_Sprint_Wins_Team.columns},
  cells=dict(
    values=GP_Sprint_Wins_Team.T.values,
    # fill_color=[GP_Sprint_Wins_Team.Color],
    align='center', font=dict(color='Black', size=9)
    ))
 )

# fig_sprint_team.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Sprint_Wins_Team.T.values
#                                 if c == "All"
#                                 else GP_Sprint_Wins_Team.loc[GP_Sprint_Wins_Team["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Sprint_Wins_Team["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_sprint_team.update_layout(
    title="Team Sprint Race Results By Position and Count",
        title_x=0.5,
        template="plotly_white",

    yaxis = dict(tickfont = dict(size=11)),
    xaxis = dict(tickfont = dict(size=11)),
    font=dict(
        family="PT Sans Narrow",
        size=11,
        color="Black"
    ),
    height=370,
    width=1400,
    title_font_family="PT Sans Narrow",
    margin=dict(l=10, r=10, t=35, b=10)
)
fig_sprint_team.show()

In [28]:
fig_sprint_team.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Team Sprint Race Results By Position and Count.html",full_html=False, include_plotlyjs='cdn')